# RDD

In [1]:
mydata = [1, 2, 2, 3, 4, 5, 5, 5, 6]

rdd = sc.parallelize(mydata)


In [2]:
rdd.collect()

In [3]:
new_RDD = rdd.map(lambda x: x * 2)

new_RDD.take(10)


In [4]:
new_RDD = rdd.filter(lambda x: x >= 4)

new_RDD.take(10)

In [5]:
new_RDD = rdd.distinct()

new_RDD.take(10)


In [6]:
rdd2 = sc.parallelize([2, 2, 3, 5, 6, 6, 7, 8, 9])
new_RDD = rdd.union(rdd2)
new_RDD.take(20)


In [7]:
rdd.reduce(lambda x, y: x + y)

In [8]:
new_RDD = rdd.intersection(rdd2)

new_RDD.take(10)


In [9]:
new_RDD = rdd.subtract(rdd2)

new_RDD.take(10)


In [10]:
new_RDD = rdd.sample(False, 0.5)

new_RDD.collect()

# PAIR RDD Transformations

In [12]:
rdd=sc.parallelize([ 
    (1, 2),
    (1, 5),
    (3, 4),
    (3, 6)])

rdd.groupByKey().collect()


In [13]:
rdd.groupByKey().map(lambda x:(x[0], list(x[1])) ).collect()

In [14]:
rdd.groupByKey().map(lambda x:(x[0], sum(x[1])) ).collect()

In [15]:
rdd.reduceByKey(lambda x, y: x + y).collect()

In [16]:
rdd.sortByKey(ascending=False).collect()

In [17]:
rdd2=sc.parallelize([(1,9)])

rdd.subtractByKey(rdd2).collect()

In [18]:
rdd.countByKey().items()

In [19]:
rdd.join(rdd2).collect()


# RDD Actions

collect()
take()
count()
top()
reduce()
first()
sum()
aggregate()

In [21]:
rdd = sc.parallelize([2, 2, 3, 5, 6, 6, 7, 8, 9, 0])

rdd.collect()


In [22]:
new_rdd=sc.parallelize([1, 2, 3, 4])

new_rdd.aggregate((0, 0), 
    (lambda x, y:(x[0] + y, x[1] + 1)), 
    (lambda x, y:(x[0] + y[0], x[1] + y[1])) )


In [23]:
# In Python
# Create an RDD of tuples (name, age)

dataRDD = sc.parallelize(
    [("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)]
    )

# Use map and reduceByKey transformations with their lambda
# expressions to aggregate and then compute average

agesRDD = (dataRDD
.map(lambda x: (x[0], (x[1], 1)))
.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
.map(lambda x: (x[0], x[1][0] / x[1][1])))

agesRDD.take(10)

In [24]:
dataRDD\
.map(lambda x: (x[0], (x[1], 1))) \
.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])).take(10)

In [25]:
from pyspark.sql.functions import avg

data_df = spark.createDataFrame([("Brooke", 20), ("Denny", 31), ("Jules", 30),
("TD", 35), ("Brooke", 25)], ["name", "age"])

# Group the same names together, aggregate their ages, and compute an average
avg_df = data_df.groupBy("name").agg(avg("age"))

avg_df.show()

In [26]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

def state_convert(code):
    return broadcastStates.value[code]

result = df.rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).toDF(columns)
result.show(truncate=False)

In [27]:
accum=spark.sparkContext.accumulator(0)
rdd=spark.sparkContext.parallelize([1,2,3,4,5])
rdd.foreach(lambda x:accum.add(x))
print(accum.value)

accuSum=spark.sparkContext.accumulator(0)
def countFun(x):
    global accuSum
    accuSum+=x
rdd.foreach(countFun)
print(accuSum.value)

accumCount=spark.sparkContext.accumulator(0)
rdd2=spark.sparkContext.parallelize([1,2,3,4,5])
rdd2.foreach(lambda x:accumCount.add(1))
print(accumCount.value)

In [28]:
sc.applicationId

# ДЗ

In [30]:
rdd = sc.parallelize([(1,2), (3,4), (3,6), (4,5), (3, 4), (1, 5), (4, 1)])


In [31]:
lines = sc.parallelize([
    "a ab abc",
    "a ac abc",
    "b b ab abc"
    ])

counts = lines.flatMap(lambda x: x.split(' ')) \
# дополнить код, чтобы получился rdd из пар (слово, частота)

output = counts.collect()

for (word, count) in output:
    print("%s: %i" % (word, count))